In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
DATAPATH = './data/games.csv'

In [3]:
games = pd.read_csv(DATAPATH)

In [4]:
games.dtypes

Unnamed: 0             int64
SEASON_ID              int64
TEAM_ID                int64
TEAM_ABBREVIATION     object
TEAM_NAME             object
GAME_ID                int64
GAME_DATE             object
MATCHUP               object
WL                    object
MIN                    int64
PTS                    int64
FGM                    int64
FGA                    int64
FG_PCT               float64
FG3M                   int64
FG3A                   int64
FG3_PCT              float64
FTM                    int64
FTA                    int64
FT_PCT               float64
OREB                   int64
DREB                   int64
REB                    int64
AST                    int64
STL                    int64
BLK                    int64
TOV                    int64
PF                     int64
PLUS_MINUS           float64
dtype: object

In [5]:
games = games.drop(['Unnamed: 0'], axis=1)

In [6]:
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22019,1610612748,MIA,Miami Heat,21900970,2020-03-11,MIA vs. CHA,L,239,98,...,0.833,9,27,36,32,8,5,14,17,-13.0
1,22019,1610612743,DEN,Denver Nuggets,21900973,2020-03-11,DEN @ DAL,L,240,97,...,0.625,3,37,40,23,6,4,15,20,-7.8
2,22019,1610612752,NYK,New York Knicks,21900969,2020-03-11,NYK @ ATL,W,264,136,...,0.735,6,41,47,32,14,9,12,23,0.0
3,22019,1610612742,DAL,Dallas Mavericks,21900973,2020-03-11,DAL vs. DEN,W,240,113,...,0.739,13,39,52,23,9,2,10,14,8.0
4,22019,1610612765,DET,Detroit Pistons,21900971,2020-03-11,DET @ PHI,L,241,106,...,0.680,6,26,32,25,10,3,8,24,-14.2


In [7]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('WL')
#     ds = tf.data.Dataset.from_tensor_slices((dataframe.values, labels.values))
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size = len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [8]:
games.pop('SEASON_ID')
games.pop('TEAM_ID')
games.pop('GAME_ID')
games.pop('TEAM_ABBREVIATION')

0        MIA
1        DEN
2        NYK
3        DAL
4        DET
        ... 
29995    BOS
29996    LAL
29997    ATL
29998    DAL
29999    DEN
Name: TEAM_ABBREVIATION, Length: 30000, dtype: object

In [9]:
feature_columns = []
obj_columns = ['TEAM_NAME', 'GAME_DATE', 'MATCHUP', 'WL']
#numeric columns
for i in obj_columns:
    games[i] = pd.Categorical(games[i])
    games[i] = games[i].cat.codes 
for header in games.columns:
    games[header] = games[header].astype('float64')
    if header != 'WL':
        feature_columns.append(feature_column.numeric_column(header))

In [10]:
games.dtypes

TEAM_NAME     float64
GAME_DATE     float64
MATCHUP       float64
WL            float64
MIN           float64
PTS           float64
FGM           float64
FGA           float64
FG_PCT        float64
FG3M          float64
FG3A          float64
FG3_PCT       float64
FTM           float64
FTA           float64
FT_PCT        float64
OREB          float64
DREB          float64
REB           float64
AST           float64
STL           float64
BLK           float64
TOV           float64
PF            float64
PLUS_MINUS    float64
dtype: object

In [11]:
games.head(n=250)

,TEAM_NAME,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42.0,2427.0,1050.0,0.0,239.0,98.0,39.0,84.0,0.464,15.0,...,0.833,9.0,27.0,36.0,32.0,8.0,5.0,14.0,17.0,-13.0
1,22.0,2427.0,465.0,0.0,240.0,97.0,39.0,83.0,0.470,14.0,...,0.625,3.0,37.0,40.0,23.0,6.0,4.0,15.0,20.0,-7.8
2,52.0,2427.0,1395.0,1.0,264.0,136.0,50.0,94.0,0.532,11.0,...,0.735,6.0,41.0,47.0,32.0,14.0,9.0,12.0,23.0,0.0
3,21.0,2427.0,434.0,1.0,240.0,113.0,42.0,92.0,0.457,12.0,...,0.739,13.0,39.0,52.0,23.0,9.0,2.0,10.0,14.0,8.0
4,23.0,2427.0,546.0,0.0,241.0,106.0,39.0,84.0,0.464,11.0,...,0.680,6.0,26.0,32.0,25.0,10.0,3.0,8.0,24.0,-14.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,47.0,2411.0,1147.0,0.0,239.0,123.0,42.0,95.0,0.442,15.0,...,0.706,7.0,35.0,42.0,16.0,4.0,6.0,11.0,22.0,-16.0
246,41.0,2411.0,961.0,0.0,241.0,97.0,37.0,89.0,0.416,7.0,...,0.727,11.0,34.0,45.0,20.0,6.0,7.0,13.0,18.0,-27.0
247,20.0,2411.0,371.0,1.0,265.0,125.0,43.0,97.0,0.443,14.0,...,0.735,13.0,40.0,53.0,25.0,8.0,6.0,12.0,20.0,6.0
248,51.0,2410.0,1346.0,1.0,239.0,115.0,46.0,98.0,0.469,17.0,...,0.462,14.0,38.0,52.0,34.0,7.0,5.0,13.0,15.0,14.0


In [12]:
games = games.dropna()

In [13]:
print(feature_columns)

[NumericColumn(key='TEAM_NAME', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='GAME_DATE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='MATCHUP', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='MIN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PTS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FGM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FGA', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FG_PCT', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FG3M', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FG3A', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key=

In [14]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [15]:
train, test = train_test_split(games, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

19196 train examples
4800 validation examples
6000 test examples


In [16]:
test

,TEAM_NAME,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
7589,88.0,1826.0,2119.0,1.0,239.0,102.0,37.0,86.0,0.430,10.0,...,0.750,10.0,32.0,42.0,17.0,12.0,5.0,22.0,20.0,8.0
28425,57.0,139.0,1640.0,1.0,240.0,92.0,37.0,86.0,0.430,3.0,...,0.833,9.0,40.0,49.0,23.0,11.0,8.0,12.0,20.0,11.0
12516,28.0,1420.0,612.0,1.0,240.0,131.0,52.0,95.0,0.547,16.0,...,0.733,10.0,36.0,46.0,29.0,10.0,6.0,15.0,19.0,8.0
24180,19.0,492.0,273.0,1.0,240.0,95.0,34.0,69.0,0.493,3.0,...,0.857,8.0,40.0,48.0,17.0,5.0,4.0,24.0,30.0,9.0
4441,12.0,2081.0,140.0,0.0,240.0,107.0,44.0,87.0,0.506,9.0,...,0.833,7.0,33.0,40.0,22.0,9.0,9.0,12.0,21.0,-8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8333,2.0,1743.0,16.0,1.0,242.0,107.0,36.0,72.0,0.500,15.0,...,0.769,6.0,40.0,46.0,22.0,9.0,6.0,18.0,23.0,17.0
14643,88.0,1239.0,2095.0,1.0,240.0,98.0,38.0,78.0,0.487,4.0,...,0.667,9.0,42.0,51.0,19.0,8.0,2.0,16.0,18.0,6.0
5840,58.0,1948.0,1676.0,0.0,240.0,83.0,33.0,95.0,0.347,3.0,...,0.636,15.0,32.0,47.0,13.0,2.0,3.0,15.0,17.0,-46.0
20151,23.0,797.0,568.0,0.0,240.0,91.0,38.0,82.0,0.463,9.0,...,0.600,6.0,23.0,29.0,28.0,7.0,4.0,16.0,18.0,-14.0


In [17]:
batch_size = 128
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)


In [18]:
def get_compiled_model():
  model = tf.keras.Sequential([
    feature_layer,
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(26, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(26, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [19]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.0001) 
mc = ModelCheckpoint('best_model.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [20]:
model = get_compiled_model()
history = model.fit(train_ds, epochs=20, callbacks=[es,mc], validation_data=val_ds)
y_pred = model.predict(val_ds).round()
y_act = np.asarray(val['WL'])
y_act = y_act.astype('float64')
y_act = np.reshape(y_act, y_pred.shape)
print('Precision= ', precision_score(y_act, y_pred , average="macro", labels=np.unique(y_pred)))
print('Recall= ', recall_score(y_act, y_pred , average="macro", labels=np.unique(y_pred)))
print('Accuracy= ', history.history['val_accuracy'][-1])
print('Loss= ', history.history['val_loss'][-1])

Train for 150 steps, validate for 38 steps
Epoch 1/20
141/150 [===========================>..] - ETA: 0s - loss: 0.6049 - accuracy: 0.8174
Epoch 00001: val_accuracy improved from -inf to 0.80063, saving model to best_model.hdf5
150/150 [==============================] - 6s 37ms/step - loss: 0.6011 - accuracy: 0.8239 - val_loss: 0.5786 - val_accuracy: 0.8006
Epoch 2/20
142/150 [===========================>..] - ETA: 0s - loss: 0.5311 - accuracy: 0.9581
Epoch 00002: val_accuracy improved from 0.80063 to 0.97062, saving model to best_model.hdf5
150/150 [==============================] - 2s 11ms/step - loss: 0.5311 - accuracy: 0.9593 - val_loss: 0.5199 - val_accuracy: 0.9706
Epoch 3/20
143/150 [===========================>..] - ETA: 0s - loss: 0.5203 - accuracy: 0.9756
Epoch 00003: val_accuracy improved from 0.97062 to 0.99104, saving model to best_model.hdf5
150/150 [==============================] - 1s 9ms/step - loss: 0.5195 - accuracy: 0.9761 - val_loss: 0.5121 - val_accuracy: 0.9910
E

In [21]:
import sys
np.set_printoptions(threshold=sys.maxsize)
y_pred = model.predict(test_ds)
for i, j in zip(y_pred, test['WL']):
    print('predicted: ', i, 'actual: ', j)

predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.9999963] actual:  1.0
predicted:  [0.99999976] actual:  1.0
predicted:  [9.5405355e-09] actual:  0.0
predicted:  [3.1524433e-08] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.9999995] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.6105241] actual:  1.0
predicted:  [0.99986804] actual:  1.0
predicted:  [0.99999964] actual:  1.0
predicted:  [3.706754e-10] actual:  0.0
predicted:  [5.994799e-10] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [2.36792e-07] actual:  0.0
predicted:  [0.9999999] actual:  1.0
predicted:  [9.395562e-09] actual:  0.0
predicted:  [8.820637e-10] actual:  0.0
predicted:  [7.848369e-09] actual:  0.0
predicted:  [6.4715766e-07] actual:  0.0
predicted:  [9.0182546e-08] actual:  0.0
predicted:  [1.7183831e-07] actual:  0.0
predicted:  [9.607191e-08] actual:  0.0
predicted:  [1

predicted:  [7.645414e-06] actual:  0.0
predicted:  [1.1499557e-10] actual:  0.0
predicted:  [0.99999833] actual:  1.0
predicted:  [0.9862476] actual:  1.0
predicted:  [1.4617975e-12] actual:  0.0
predicted:  [4.1941967e-05] actual:  0.0
predicted:  [1.912483e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.22556e-07] actual:  0.0
predicted:  [1.9645424e-05] actual:  0.0
predicted:  [4.1498513e-08] actual:  0.0
predicted:  [9.719455e-06] actual:  0.0
predicted:  [3.3789363e-06] actual:  0.0
predicted:  [4.0876028e-08] actual:  0.0
predicted:  [0.99999547] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [6.62918e-08] actual:  0.0
predicted:  [0.9999999] actual:  1.0
predicted:  [5.3842527e-06] actual:  0.0
predicted:  [0.00011553] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [0.9999951] actual:  1.0
predicted:  [0.99999964] actual:  1.0
predicted:  [1.1807488e-08] actual:  0.0
predicted:  [1.4482712e-08] actual:  0.0
predicted

predicted:  [1.] actual:  1.0
predicted:  [0.9999989] actual:  1.0
predicted:  [5.0303026e-07] actual:  0.0
predicted:  [6.6422444e-11] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [0.00727955] actual:  0.0
predicted:  [3.3223388e-10] actual:  0.0
predicted:  [0.99999976] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.9999999] actual:  1.0
predicted:  [1.0470102e-11] actual:  0.0
predicted:  [0.9999999] actual:  1.0
predicted:  [0.9999999] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.105705e-07] actual:  0.0
predicted:  [0.99950516] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [6.6438154e-11] actual:  0.0
predicted:  [1.1559246e-09] actual:  0.0
predicted:  [3.453635e-08] actual:  0.0
predicted:  [0.9999255] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.1165366e-07] actual:  0.0
predicted:  [0.9999957] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.9992048] actual:  1.0
predicted:  [1.23060

predicted:  [3.8179067e-08] actual:  0.0
predicted:  [0.00697615] actual:  0.0
predicted:  [0.9999963] actual:  1.0
predicted:  [0.9999999] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [7.454667e-08] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [0.9999932] actual:  1.0
predicted:  [2.575174e-07] actual:  0.0
predicted:  [0.99997914] actual:  1.0
predicted:  [1.4550791e-09] actual:  0.0
predicted:  [2.091013e-10] actual:  0.0
predicted:  [0.99831945] actual:  1.0
predicted:  [0.99999917] actual:  1.0
predicted:  [1.8134587e-07] actual:  0.0
predicted:  [3.709284e-11] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [2.287325e-07] actual:  0.0
predicted:  [6.586109e-10] actual:  0.0
predicted:  [9.5704294e-08] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [0.92429984] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [2.545905e-08] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [5.4099236e-10] actual:  0.0
predicted:  [5.0128945e-10] actual

predicted:  [3.7241986e-07] actual:  0.0
predicted:  [1.423276e-10] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [0.9757213] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [5.3742e-12] actual:  0.0
predicted:  [5.9884854e-08] actual:  0.0
predicted:  [6.809807e-11] actual:  0.0
predicted:  [0.9999999] actual:  1.0
predicted:  [0.00026737] actual:  0.0
predicted:  [2.9341223e-07] actual:  0.0
predicted:  [0.99999905] actual:  1.0
predicted:  [1.8504254e-05] actual:  0.0
predicted:  [1.7323753e-12] actual:  0.0
predicted:  [0.99999666] actual:  1.0
predicted:  [0.9999999] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [5.670908e-08] actual:  0.0
predicted:  [5.595742e-11] actual:  0.0
predicted:  [0.9999995] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.9999964] actual:  1.0
predicted:  [0.9999857] actual:  1.0
predicted:  [0.99944144] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.99999845] actual:  1.0
predi

...

predicted:  [0.9999999] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.9999999] actual:  1.0
predicted:  [0.99998856] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [2.6690634e-07] actual:  0.0
predicted:  [0.96369416] actual:  1.0
predicted:  [0.01140782] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.4297795e-09] actual:  0.0
predicted:  [1.7278552e-11] actual:  0.0
predicted:  [1.6547932e-10] actual:  0.0
predicted:  [1.4745525e-06] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [0.860368] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.9999994] actual:  1.0
predicted:  [0.9999975] actual:  1.0
predicted:  [0.99999726] actual:  1.0
predicted:  [5.0978877e-11] actual:  0.0
predicted:  [3.8908015e-09] actual:  0.0
predicted:  [0.00054493] actual:  0.0
predicted:  [1.2811964e-09] actual:  0.0
predicted:  [0.9999982] actual:  1.0
predicted:  [1.] actu

predicted:  [0.9999994] actual:  1.0
predicted:  [7.514061e-06] actual:  0.0
predicted:  [4.1658706e-05] actual:  0.0
predicted:  [3.5084307e-07] actual:  0.0
predicted:  [6.806372e-09] actual:  0.0
predicted:  [1.0054998e-09] actual:  0.0
predicted:  [9.0442445e-09] actual:  0.0
predicted:  [2.4074462e-08] actual:  0.0
predicted:  [2.4054416e-06] actual:  0.0
predicted:  [1.3810245e-09] actual:  0.0
predicted:  [0.99999976] actual:  1.0
predicted:  [0.9999995] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.99998415] actual:  1.0
predicted:  [0.9999918] actual:  1.0
predicted:  [0.99999964] actual:  1.0
predicted:  [4.68207e-13] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [0.9999963] actual:  1.0
predicted:  [4.303737e-08] actual:  0.0
predicted:  [1.7301895e-08] actual:  0.0
predicted:  [0.9999969] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.7960573e-08] actual:  0.0
predicted:  [0.99999976] actual:  1.0
predicted:  [4.7267677e-06] actual:  0.0
pred

predicted:  [1.6365075e-09] actual:  0.0
predicted:  [0.9994808] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.00020318] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [3.01791e-07] actual:  0.0
predicted:  [0.9999999] actual:  1.0
predicted:  [0.99536407] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.0003469e-08] actual:  0.0
predicted:  [1.2466533e-08] actual:  0.0
predicted:  [2.2385455e-11] actual:  0.0
predicted:  [0.99999845] actual:  1.0
predicted:  [0.9999989] actual:  1.0
predicted:  [5.310604e-09] actual:  0.0
predicted:  [1.4446055e-05] actual:  0.0
predicted:  [5.1197104e-09] actual:  0.0
predicted:  [7.3116904e-09] actual:  0.0
predicted:  [4.887579e-14] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [3.415224e-11] actual:  0.0
predicted:  [6.91939e-06] actual:  0.0
predicted:  [0.9998957] actual:  1.0
predicted:  [0.99999774] actual:  1.0
predicted:  [2.0736792e-11] actual:  0.0
predicted:  [1.] actual:

predicted:  [0.99999976] actual:  1.0
predicted:  [4.8773945e-09] actual:  0.0
predicted:  [1.2638282e-09] actual:  0.0
predicted:  [4.2568415e-13] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.6070171e-09] actual:  0.0
predicted:  [7.6096234e-07] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [2.0828907e-08] actual:  0.0
predicted:  [1.0018091e-09] actual:  0.0
predicted:  [6.3018986e-11] actual:  0.0
predicted:  [2.650111e-08] actual:  0.0
predicted:  [1.1336501e-07] actual:  0.0
predicted:  [0.99964046] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [5.0185363e-11] actual:  0.0
predicted:  [0.97579205] actual:  1.0
predicted:  [0.00062197] actual:  0.0
predicted:  [3.5472962e-07] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [2.1685217e-08] actual:  0.0
predicted:  [0.99998355] actual:  1.0
predicted:  [3.6345364e-06] actual:  0.0
predicted:  [6.6984343e-09] actual:  0.0
predicted:  [0.99810684] actual:  1.0
predicted:  [8.732506e-09] actual:  0.

predicted:  [0.99999976] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [4.2091892e-07] actual:  0.0
predicted:  [0.4850482] actual:  1.0
predicted:  [3.2944832e-15] actual:  0.0
predicted:  [0.9999893] actual:  1.0
predicted:  [0.99999857] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.6560042e-08] actual:  0.0
predicted:  [1.6465248e-11] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.1740603e-05] actual:  0.0
predicted:  [0.000252] actual:  0.0
predicted:  [2.185198e-06] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [0.9999999] actual:  1.0
predicted:  [1.24433375e-17] actual:  0.0
predicted:  [6.622084e-10] actual:  0.0
predicted:  [0.9999918] actual:  1.0
